# Q3 - R section

In this section, we use R to perform differential expression analysis for feature selection. We are going to do this analysis for both machines

In my opinion, differential expression (DE) analysis is the most efficient feature selection method in this case because it not only considers the variance or mutual information of features but also how they are expressed across different classes. Therefore, I believe it will help us identify the best features (DEGs) for classification.

## First Machine ##

In [23]:
# Import needed libraries
library(limma)
library(edgeR)

In [24]:
# Load Data
normal_counts <- read.csv("train_normal_counts.csv")
meta_data <- read.csv("train_meta_data.csv")

In [25]:
head(normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.820135,6.5462994,6.6040504,6.480745,6.550016,6.5692529,5.9652146,6.436726,6.3378589,6.3677048,⋯,6.85952138,6.3703867,6.6352203,6.693334,6.26633823,6.6089237,6.34423435,6.296354,6.485184,5.965571
2,-1.060061,0.5821648,-0.8650363,-1.083676,-1.222374,0.7672549,-0.2056643,1.225274,0.3287041,-0.9022011,⋯,0.03594184,-0.5651604,-0.4658174,1.603149,0.08898988,-0.7573986,-0.02658513,-2.568864,-0.486609,1.612375
3,4.388400,3.7520898,4.3514891,4.361634,4.534941,4.1504701,3.1093774,4.104779,4.6099902,4.1480104,⋯,4.12340468,4.7392966,4.3778794,4.473400,4.31945643,4.2744503,4.17931917,4.381449,4.331904,4.133821
4,4.080172,4.6451746,4.0721368,4.313540,4.370763,4.1660389,4.8092861,4.411547,4.3253607,4.5060579,⋯,4.76204644,4.1208213,4.2298584,4.755317,4.38806704,4.0425768,4.43879576,4.365810,4.090491,4.111056
5,2.564430,3.8408991,3.1431376,3.120196,3.512952,3.7570108,3.9527651,3.099743,3.3085263,3.9820712,⋯,3.95559973,2.9326175,3.6972213,4.027257,3.97372734,2.1264084,3.81471713,2.965115,2.697188,4.150662
6,3.552685,3.2010747,4.0374758,1.941859,2.517867,3.2536535,2.7857226,3.357725,1.9136666,2.2021356,⋯,2.72896408,2.8246104,2.2057995,1.576431,2.97651515,3.4239312,2.27774891,3.408416,3.630074,2.975845


In [26]:
dim(normal_counts)

[1] 17396   134

In [27]:
head(meta_data)

,Simplified_class,class
,<chr>,<chr>
1,Normal,Normal
2,Advanced_fibrosis,Fibrosis
3,Normal,Normal
4,Normal,Normal
5,Non_advanced_Fibrosis,Fibrosis
6,Normal,Normal


In [28]:
labels <- factor(meta_data$class)

In [29]:
print(labels)

  [1] Normal   Fibrosis Normal   Normal   Fibrosis Normal   Fibrosis Fibrosis
  [9] Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis
 [17] Normal   Normal   Normal   Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [25] Fibrosis Fibrosis Normal   Fibrosis Normal   Fibrosis Fibrosis Fibrosis
 [33] Fibrosis Normal   Normal   Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis
 [41] Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis
 [49] Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis Normal   Fibrosis
 [57] Fibrosis Fibrosis Normal   Normal   Normal   Fibrosis Fibrosis Fibrosis
 [65] Normal   Normal   Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [73] Normal   Fibrosis Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [81] Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis
 [89] Fibrosis Fibrosis Fibrosis Normal   Normal   Fibrosis Fibrosis Fibrosis
 [97] Fibrosis Fibrosis Fibrosis Fibrosis Normal   Normal   Fibr

Let's perform DE analysis

In [30]:
# Create a design matrix
design <- model.matrix(~0 + labels)
colnames(design) <- levels(labels)

In [31]:
fit <- lmFit(normal_counts, design)

In [32]:
contrast.matrix <- makeContrasts(
    Fibrosis_vs_Normal = `Fibrosis` - Normal,
    levels = design
)

# Apply contrasts to the fit
fit2 <- contrasts.fit(fit, contrast.matrix)

# Empirical Bayes moderation to get p-values
fit2 <- eBayes(fit2)

Now, we are going to extract the DEGs for Fibrosis vs Normal pair and save them

In [33]:
# Get the top DEGs for the Fibrosis vs Normal comparison
top_genes_fib_vs_norm <- topTable(fit2, coef = "Fibrosis_vs_Normal", adjust.method = "BH", number = Inf)

# View the top DEGs
head(top_genes_fib_vs_norm)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6969,0.4767457,4.207436,11.50608,9.952564e-22,1.731348e-17,38.77566
17075,0.5206691,7.265926,11.08760,1.140214e-20,7.320483e-17,36.39218
4419,0.8110163,7.074491,11.04250,1.482760e-20,7.320483e-17,36.13539
10970,0.5251446,5.361546,11.01330,1.757697e-20,7.320483e-17,35.96911
7725,0.6071783,4.957512,10.97299,2.222858e-20,7.320483e-17,35.73958
1776,0.7526904,4.613934,10.92701,2.905256e-20,7.320483e-17,35.47785


In [34]:
write.csv(top_genes_fib_vs_norm, "DEGs_Fibrosis_from_Normal.csv")

We have filtered the top 300 DEGs for each pair. The choice of 𝑘=300 appears to be optimized based on our greedy search, which has not been included in this notebook.

In [35]:
filtered_genes_fib_vs_norm <- top_genes_fib_vs_norm[1:300,]

# View filtered DEGs
head(filtered_genes_fib_vs_norm)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6969,0.4767457,4.207436,11.50608,9.952564e-22,1.731348e-17,38.77566
17075,0.5206691,7.265926,11.08760,1.140214e-20,7.320483e-17,36.39218
4419,0.8110163,7.074491,11.04250,1.482760e-20,7.320483e-17,36.13539
10970,0.5251446,5.361546,11.01330,1.757697e-20,7.320483e-17,35.96911
7725,0.6071783,4.957512,10.97299,2.222858e-20,7.320483e-17,35.73958
1776,0.7526904,4.613934,10.92701,2.905256e-20,7.320483e-17,35.47785


These top 300 DEGs are biologically meaningful in addition to their role in computationally classifying the data. They are likely genes whose expression changes significantly when transitioning from Normal to Fibrosis class. These genes are probably among the most correlated with the class labels, though they are not necessarily causal genes. This change in class label may have a substantial impact on their expression, potentially affecting their associated pathways or other related biological processes.

In [36]:
genes_fib_vs_norm_names <- rownames(filtered_genes_fib_vs_norm)

In [37]:
common_genes <- intersect(rownames(normal_counts), genes_fib_vs_norm_names)
selected_normal_counts <- normal_counts[common_genes, ]
head(selected_normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,4.5895546,5.48216905,5.013154,5.0660709,4.655817,4.299078,4.752957,5.4095139,4.99377681,4.5285302,⋯,4.8535651,4.5671727,4.9851582,5.2347762,4.9007545,4.9230866,5.0044999,5.058062,5.0024535,4.221450
17,5.4498461,5.62655895,5.668470,5.8249308,5.645748,5.172477,5.550495,5.4456837,5.82529905,5.6696779,⋯,5.5740661,5.2507565,6.0089895,6.0142514,5.9716329,5.7074875,5.7846679,5.655138,5.9198271,5.086468
67,5.5138387,6.02747535,6.073207,5.8621856,5.755947,4.997887,5.706940,5.7806501,6.01986604,5.8642174,⋯,5.7565851,5.4751801,5.9818563,5.8731974,5.8628355,5.8325646,5.7705851,5.658351,6.0170439,5.071807
278,5.5299023,5.84627227,5.553583,5.3592673,5.774549,4.962662,5.201300,5.5002050,5.78205099,5.4935472,⋯,5.8620666,5.0386033,5.6652311,5.7811395,5.7451593,5.4380615,5.5701694,5.558701,5.7338219,5.220121
301,5.7134081,6.17141469,5.919896,6.0819486,5.670238,5.175044,5.884826,5.8896988,6.27685502,5.6307392,⋯,5.9347951,5.7372670,6.2334328,6.2235198,5.9362710,6.1094388,5.7242487,6.160191,6.0321724,5.201384
310,0.3184508,0.09673801,1.033084,0.3568963,0.362589,1.189082,1.777058,0.8940684,0.09737859,0.5484603,⋯,0.2165141,0.9837329,0.1862593,0.3895531,0.5744167,0.8726518,0.4208739,1.574094,0.6045389,2.394029


In [38]:
dim(selected_normal_counts)

[1] 300 134

We extracted a subset from the data based on selected features. Let's save it and continue with the second machine.

In [39]:
write.csv(selected_normal_counts, "subset_data1.csv")
write.csv(meta_data, "meta_data1.csv")

## Second Machine ##

In [40]:
meta_data <- subset(meta_data, Simplified_class != 'Normal')

In [41]:
head(meta_data)

,Simplified_class,class
,<chr>,<chr>
2,Advanced_fibrosis,Fibrosis
5,Non_advanced_Fibrosis,Fibrosis
7,Advanced_fibrosis,Fibrosis
8,Non_advanced_Fibrosis,Fibrosis
9,Non_advanced_Fibrosis,Fibrosis
11,Advanced_fibrosis,Fibrosis


In [42]:
dim(meta_data)

[1] 90  2

In [43]:
rownames(meta_data)

[1] "2"   "5"   "7"   "8"   "9"   "11"  "12"  "13"  "14"  "15"  "16"  "21" 
[13] "22"  "23"  "24"  "25"  "26"  "28"  "30"  "31"  "32"  "33"  "36"  "37" 
[25] "38"  "39"  "40"  "41"  "42"  "43"  "44"  "45"  "46"  "48"  "49"  "50" 
[37] "51"  "52"  "54"  "56"  "57"  "58"  "62"  "63"  "64"  "67"  "69"  "70" 
[49] "71"  "72"  "74"  "75"  "77"  "78"  "79"  "80"  "81"  "83"  "84"  "85" 
[61] "86"  "88"  "89"  "90"  "91"  "94"  "95"  "96"  "97"  "98"  "99"  "100"
[73] "103" "104" "107" "108" "109" "111" "112" "115" "118" "119" "120" "122"
[85] "123" "124" "125" "127" "128" "129"

In [44]:
fibrosis_normal_counts <- normal_counts[as.integer(rownames(meta_data))]

In [45]:
head(fibrosis_normal_counts)

,DLDR_0081,DLDR_0130,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0095,DLDR_0097,DLDR_0086,DLDR_0149,DLDR_0044,⋯,DLDR_0166,DLDR_0089,DLDR_0141,DLDR_0111,DLDR_0136,DLDR_0134,DLDR_0175,DLDR_0087,DLDR_0155,DLDR_0092
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6.5462994,6.550016,5.9652146,6.436726,6.3378589,6.432976,6.268976,6.7395306,6.7404826,5.689253,⋯,6.8765094,6.9340665,6.6000053,6.4707867,6.0799502,6.1192447,6.85952138,6.6352203,6.693334,6.26633823
2,0.5821648,-1.222374,-0.2056643,1.225274,0.3287041,-1.235636,-2.378483,0.2980298,-0.8093479,-1.246025,⋯,-0.8545833,0.1054011,0.9989488,0.2178724,-0.3480662,-0.4492898,0.03594184,-0.4658174,1.603149,0.08898988
3,3.7520898,4.534941,3.1093774,4.104779,4.6099902,4.061103,4.097251,4.2049204,4.6702399,4.245828,⋯,4.2014221,3.8919974,4.8759379,4.6599428,4.5581065,4.3102366,4.12340468,4.3778794,4.473400,4.31945643
4,4.6451746,4.370763,4.8092861,4.411547,4.3253607,4.620430,4.191373,4.2132597,4.4935629,4.706380,⋯,4.3444346,4.1351484,4.9302925,3.8586881,4.4179435,4.4967901,4.76204644,4.2298584,4.755317,4.38806704
5,3.8408991,3.512952,3.9527651,3.099743,3.3085263,3.682309,3.643885,3.3367102,3.2119626,3.467163,⋯,3.4332830,2.2645997,3.3930267,3.0252273,3.4912316,3.1615746,3.95559973,3.6972213,4.027257,3.97372734
6,3.2010747,2.517867,2.7857226,3.357725,1.9136666,1.506867,2.145079,2.9741402,1.5927505,3.059160,⋯,2.6897372,2.7063051,1.6442839,2.9811721,2.5076855,2.9158915,2.72896408,2.2057995,1.576431,2.97651515


In [46]:
dim(fibrosis_normal_counts)

[1] 17396    90

In [47]:
labels <- factor(meta_data$Simplified_class)

In [48]:
print(labels)

 [1] Advanced_fibrosis     Non_advanced_Fibrosis Advanced_fibrosis    
 [4] Non_advanced_Fibrosis Non_advanced_Fibrosis Advanced_fibrosis    
 [7] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[10] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[13] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[16] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[19] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[22] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[25] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[28] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[31] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[34] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[37] Non_advanced_Fibrosis Advanced_fibrosis     Non_advanced_Fibrosis
[40] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[43] N

Let's perform DE analysis

In [49]:
# Create a design matrix
design <- model.matrix(~0 + labels)
colnames(design) <- levels(labels)

In [50]:
fit <- lmFit(fibrosis_normal_counts, design)

In [51]:
contrast.matrix <- makeContrasts(
    AdvancedFibrosis_vs_Fibrosis = `Advanced_fibrosis` - Non_advanced_Fibrosis,
    levels = design
)

# Apply contrasts to the fit
fit2 <- contrasts.fit(fit, contrast.matrix)

# Empirical Bayes moderation to get p-values
fit2 <- eBayes(fit2)

Now, we are going to extract the DEGs for Advanced Fibrosis vs Non-Advanced Fibrosis pair and save them

In [53]:
# Get the top DEGs for the Advanced Fibrosis vs Fibrosis comparison
top_genes_adv_vs_fib <- topTable(fit2, coef = "AdvancedFibrosis_vs_Fibrosis", adjust.method = "BH", number = Inf)

# View the top DEGs
head(top_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9231954,8.604058,2.266513e-13,3.942826e-09,19.90756
4770,0.482585,4.4402232,7.784892,1.119579e-11,4.824356e-08,16.20400
12710,1.181496,-0.2294500,7.774583,1.175495e-11,4.824356e-08,16.15769
673,1.485822,1.1041118,7.739811,1.385410e-11,4.824356e-08,16.00156
3296,1.594761,2.1489644,7.739625,1.386628e-11,4.824356e-08,16.00073
12060,1.675825,-0.0107199,7.562489,3.195875e-11,9.265907e-08,15.20726


In [54]:
write.csv(top_genes_adv_vs_fib, "DEGs_Advanced_from_NonAdvanced.csv")

We have filtered the top 300 DEGs for each pair. The choice of 𝑘=300 appears to be optimized based on our greedy search, which has not been included in this notebook.

In [55]:
filtered_genes_adv_vs_fib <- top_genes_adv_vs_fib[1:300,]

# View filtered DEGs
head(filtered_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9231954,8.604058,2.266513e-13,3.942826e-09,19.90756
4770,0.482585,4.4402232,7.784892,1.119579e-11,4.824356e-08,16.20400
12710,1.181496,-0.2294500,7.774583,1.175495e-11,4.824356e-08,16.15769
673,1.485822,1.1041118,7.739811,1.385410e-11,4.824356e-08,16.00156
3296,1.594761,2.1489644,7.739625,1.386628e-11,4.824356e-08,16.00073
12060,1.675825,-0.0107199,7.562489,3.195875e-11,9.265907e-08,15.20726


These top 300 DEGs are biologically meaningful in addition to their role in computationally classifying the data. They are likely genes whose expression changes significantly when transitioning from Non-Advanced Fibrosis to Advanced Fibrosis class. These genes are probably among the most correlated with the class labels, though they are not necessarily causal genes. This change in class label may have a substantial impact on their expression, potentially affecting their associated pathways or other related biological processes.

In [56]:
genes_adv_vs_fib_names <- rownames(filtered_genes_adv_vs_fib)

In [57]:
common_genes <- intersect(rownames(fibrosis_normal_counts), genes_adv_vs_fib_names)
selected_normal_counts <- fibrosis_normal_counts[common_genes, ]
head(selected_normal_counts)

,DLDR_0081,DLDR_0130,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0095,DLDR_0097,DLDR_0086,DLDR_0149,DLDR_0044,⋯,DLDR_0166,DLDR_0089,DLDR_0141,DLDR_0111,DLDR_0136,DLDR_0134,DLDR_0175,DLDR_0087,DLDR_0155,DLDR_0092
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
57,0.5821648,-0.2894877,-1.138550,-4.060128,-2.1042553,0.01812021,-0.05655446,-1.903604,-1.7422337,-1.039574,⋯,-1.954119,-0.6315645,-4.286453,0.4935068,-2.0218380,-2.6509237,-2.285986,-0.385647,-3.580073,0.3172589
209,7.0012221,7.2057963,6.750193,7.459017,7.3791575,7.11479286,7.02026114,7.433126,7.5770278,7.142655,⋯,7.233940,7.3864811,7.616299,6.8790879,7.2807466,7.0989458,7.339114,7.366168,7.612425,7.3409794
232,7.1692726,6.9989939,7.278919,7.887874,7.7238812,7.41458689,7.02026114,7.363651,7.6123162,7.838375,⋯,7.585871,6.6216264,7.817818,7.0580706,7.7930878,7.4547230,7.755217,7.485884,7.339038,6.9069785
275,2.5296974,1.4679420,2.031375,2.048397,0.5280129,1.13001309,1.70898029,0.418324,0.7756146,1.767781,⋯,1.777685,1.9264309,1.644284,1.9923127,0.5047078,0.7550687,1.083248,2.055951,1.374123,1.9022214
297,7.7592284,7.4394046,7.481170,7.556880,8.3222192,8.61647293,8.95075319,8.212480,7.6611657,6.947216,⋯,7.898745,7.7979585,7.851498,8.6013493,7.1909768,7.5771735,7.462766,8.581307,7.940709,8.7308868
390,8.6335680,8.2483778,8.367261,8.726754,7.2475674,7.84261708,8.49419233,8.105105,7.4192417,8.842356,⋯,8.417658,8.0937070,7.483798,8.7469947,7.6486497,7.7942982,8.077053,8.348605,7.768471,8.3494726


In [58]:
dim(selected_normal_counts)

[1] 300  90

We extracted a subset from the data based on selected features. Let's save it and continue the analysis in Python Jupyter Notebook

In [59]:
write.csv(selected_normal_counts, "subset_data2.csv")
write.csv(meta_data, "meta_data2.csv")